In [350]:
from torch_geometric.datasets import KarateClub
import networkx as nx
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset


In [351]:
graph = nx.karate_club_graph()
X = nx.to_numpy_array(graph)
position = nx.spring_layout(graph, seed=55)
N = len(X)
X

array([[0., 4., 5., ..., 2., 0., 0.],
       [4., 0., 6., ..., 0., 0., 0.],
       [5., 6., 0., ..., 0., 2., 0.],
       ...,
       [2., 0., 0., ..., 0., 4., 4.],
       [0., 0., 2., ..., 4., 0., 5.],
       [0., 0., 0., ..., 4., 5., 0.]])

In [354]:
labels = []
for i in graph.nodes:
    club_names = graph.nodes[i]['club']
    labels.append(1 if club_names== 'Officer' else 0)
Y = np.array(labels)
X = torch.FloatTensor(X)
y = torch.LongTensor(Y)
Y.shape

(34,)

In [355]:
# Split the dataset into training and testing sets
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

In [356]:
# Create DataLoader for training and testing sets
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

In [357]:
class KarateClubClassifier(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(KarateClubClassifier, self).__init__()
        self.layers = []
        for i, hidden_size in enumerate(hidden_sizes):
            if i == 0:
                self.layers.append(nn.Linear(input_size, hidden_size))
            else:
                self.layers.append(nn.Linear(hidden_sizes[i-1], hidden_size))
            self.layers.append(nn.ReLU())
        self.layers.append(nn.Linear(hidden_sizes[-1], output_size))
        self.model = nn.Sequential(*self.layers)

    def forward(self, x):
        return self.model(x)


In [390]:
input_size = X.shape[1]
layer_result = []

def layer_run(layer_size):
    hidden_sizes = [64 for i in range(layer_size)] 
    output_size = 2
    model = KarateClubClassifier(input_size, hidden_sizes, output_size)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    for epoch in range(70):
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('layer_size: {} , Accuracy:  {} %'.format(layer_size,100 * correct / total))
    layer_result.append(100 * correct / total)



### Section A&B: 2-20 layers

In [391]:
for i in range(1, 20):
    layer_run(i)

layer_size: 1 , Accuracy:  100.0 %
layer_size: 2 , Accuracy:  100.0 %
layer_size: 3 , Accuracy:  100.0 %


layer_size: 4 , Accuracy:  100.0 %
layer_size: 5 , Accuracy:  100.0 %
layer_size: 6 , Accuracy:  100.0 %
layer_size: 7 , Accuracy:  100.0 %
layer_size: 8 , Accuracy:  71.42857142857143 %
layer_size: 9 , Accuracy:  100.0 %
layer_size: 10 , Accuracy:  100.0 %
layer_size: 11 , Accuracy:  100.0 %
layer_size: 12 , Accuracy:  57.142857142857146 %
layer_size: 13 , Accuracy:  85.71428571428571 %
layer_size: 14 , Accuracy:  85.71428571428571 %
layer_size: 15 , Accuracy:  28.571428571428573 %
layer_size: 16 , Accuracy:  57.142857142857146 %
layer_size: 17 , Accuracy:  57.142857142857146 %
layer_size: 18 , Accuracy:  42.857142857142854 %
layer_size: 19 , Accuracy:  71.42857142857143 %


### Section C:

by increasing the number of layers, the accuracy of the model increases. However, the accuracy of the model is not always increasing. For example, when the number of layers is 7, the accuracy of the model is 100, but when the number of layers is 8, the accuracy of the model is 71. This is because the model is overfitting. for high number of layers, the model is overfitting and the accuracy of the model is decreasing and model going to be unstable.